# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment,ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import os
import joblib
from endpoint import ConsumeEndPoint
from logs import LogService


In [ ]:
ws = Workspace.from_config()

experiment_name = 'capstone_project'

experiment=Experiment(ws, experiment_name)


In [ ]:
cpu_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.05, evaluation_interval=3)  

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--max_iter": choice(50, 75),
        "--C": uniform(.3, .6)
       
    })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory= './',entry_script='train.py',
compute_target = compute_target,
pip_packages=['argparse','pandas','numpy'] )

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                hyperparameter_sampling=param_sampling,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs = 4     
                                )

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config,show_output=True)

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_best_model = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
#TODO: Save the best model
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

joblib.dump(hyperdrive_best_model, 'outputs/hyperdrive_best_model.joblib')

import pickle
pickle.dump(hyperdrive_best_model, 'outputs/hyperdrive_best_model.pkl')

In [ ]:
registered_model = Model.register(workspace=ws,model_path='outputs/hyperdrive_best_model.joblib', model_name='capstone_hyperdrive_best_model')

In [ ]:
# Create the environment
#env = Environment(name="myenv")

myenv = Environment.get(workspace, "AzureML-Minimal").clone('myenv')

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

# conda_dep = CondaDependencies()

# # Define the packages needed by the model and scripts
# conda_dep.add_conda_package("numpy")
# conda_dep.add_conda_package("scikit-learn")
# # You must list azureml-defaults as a pip dependency
# conda_dep.add_pip_package("azureml-defaults")
# conda_dep.add_pip_package("azureml-core")
# # Adds dependencies to PythonSection of myenv
# myenv.python.conda_dependencies=conda_dep

In [ ]:
# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                       auth_enabled=True)

In [ ]:
service = Model.deploy(ws, "hyerdrivewebservice", [registered_model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

In [ ]:
TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

swagger_uri = service.swagger_uri

primary, secondary = service.get_keys()

print(service.scoring_uri)

print(service.swagger_uri)

ConsumeEndPoint.SendRequest(scoring_uri,primaryKey)

In [ ]:
LogService.Log("hyerdrivewebservice")

In [ ]:
service.delete()

In [ ]:
# try:
#     compute_target.delete()
# except ComputeTargetException:
#     print("cpu-cluster Not Found")